# Dynamic Programming
- Can be used to "plan" i.e. $\pi^*$ or $V^*$ find when MDP $(S, P, d, R, A)$ is fully known
    - 1) Policy Evaluation / Prediction - For any $\pi$ find $V^\pi(s)$ and $Q^\pi(s,a)$ 
    - 2) Policy Iteration / Control - Find $\pi^*$ by $\pi_{k+1}(s) = argmax_{a} Q^{\pi_{k}}(s,a)$
    - 3) Value Iteration / Control -  Find $V^*$ direclty via $V_{k+1}(s) = max_{a} \left( R^a_{s} + d * \sum_{s'}P^a_{s,s'}V(s') \right)$

In [10]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (30, 10)
plt.rcParams['font.size']=20

# 1) Policy Evaluation


- Given a MDP $(S, P, d, R, A)$ for any arbitrary policy $\pi$ we can find $V^\pi$
- Guess $V^\pi_{0}$, and calculate $V^\pi_{1}$, $V^\pi_{2}$, ... 
- for each state $s$, $V^\pi_{k+1}(s) = \sum_{a} \pi(a|s) \left( R^a_{s} + \sum_{s'} P^a_{s,s'} V^\pi_{k}(s')\right)$
- Contraction mapping theorem says that RHS is a contraction map and will converge on repeated iterations

In [11]:
# Simple MDP
S = np.array([0, 2]) 
A = np.array([100, 200])
P = np.array([[[0.5, 0.5],   # P given a=0
              [0.8, 0.2]], 
     
             [[0.2, 0.8],   # P given a=0
              [0.4, 0.6]]] 
 
             )
R = np.array([[10, 90], 
               [5, 30]])
d = 0.95
s0 = 0
pi = np.array([[0.5, 0.5], 
               [0.5, 0.5]])
T = 100

def discountedSum(R, d):
    sum = 0
    for i, r in enumerate(R):
        sum += R[i] * (d ** i)
    return sum

# Histories
HS = np.zeros(T+1).astype(int)
HR = np.zeros(T).astype(int)
HA = np.zeros(T).astype(int)
HG = np.zeros(T).astype(int)
HV = np.zeros(T).astype(int)
HS[0] = s0

In [12]:
# Policy Evaluation
N = 1000 # iterations
V_new = np.zeros(S.shape[0])
V_old = np.ones(S.shape[0])
for i_n in range(N):
    for i_s, s in enumerate(S):
        V_new[i_s] = np.dot(pi[i_s], R[:, i_s] + d * np.dot(P[:, i_s, :], V_old))
    V_old = V_new.copy()
print(V_new)

[673.93939394 716.36363636]


In [13]:
# Monte Carlo
N = 500 
V = np.empty((T, S.shape[0]))
Q = np.empty((T, A.shape[0], S.shape[0]))
G_ = np.empty((T, S.shape[0], N))
for i_s, s0 in enumerate(S):
    for i_n in range(N):
        HS = np.zeros(T+1).astype(int)
        HR = np.zeros(T).astype(int)
        HA = np.zeros(T).astype(int)
        HG = np.zeros(T).astype(int)
        HV = np.zeros(T).astype(int)
        HS[0] = s0
        for t in range(T):
            HA[t] = np.random.choice(A, p = pi[np.where(S == HS[t])][0])
            HS[t+1] = np.random.choice(S, p = P[np.where(A == HA[t]), np.where(S == HS[t]), :][0][0])
            HR[t] = R[np.where(A == HA[t]), np.where(S == HS[t])]
        for t in range(0, T, 1):
            HG[T-1-t] = discountedSum(HR[T-1-t:T], d)
        G_[:, i_s, i_n] = HG
    for t in range(T-1):
        V[t, i_s] = np.mean(G_[t, i_s, :])           
print(V[0, :])

[671.946 707.104]


* MDP 2

In [14]:
S = np.array([0, 1, 2]) # three states
A = np.array([100, 200, 300]) # three decisions
P = np.array([[[0.5, 0.4, 0.1],   # P given a=0
              [0.8, 0.1, 0.1],
              [0.2, 0.1, 0.7]], 
     
             [[0.2, 0.2, 0.6],   # P given a=0
              [0.4, 0.3, 0.3],
              [0.5, 0.4, 0.1]], 
 
             [[0.1, 0.7, 0.2],   # P given a=0
              [0.05, 0.9, 0.05],
              [0.3, 0.2, 0.5]],
             ])
R = np.array([[10, 20, 90], 
               [5, 30, 80],
              [4, 4, 110]])
d = 0.95
s0 = 0
pi = np.array([[0.3, 0.5, 0.2], 
               [0.7, 0.2, 0.1], 
              [0.3, 0.4, 0.3]])

In [15]:
# Policy Evaluation
N = 2000
V_new = np.zeros(S.shape[0])
V_old = np.ones(S.shape[0])
print(V_old)
for i_n in range(N):
    for i_s, s in enumerate(S):
        V_new[i_s] = np.dot(pi[i_s], R[:, i_s] + d * np.dot(P[:, i_s, :], V_old))
    V_old = V_new.copy()
print(V_new)

[1. 1. 1.]
[715.45797971 710.41391389 804.21464968]


In [16]:
# Monte Carlo Check
N = 2000 # histories
V = np.empty((T, S.shape[0]))
Q = np.empty((T, A.shape[0], S.shape[0]))
G_ = np.empty((T, S.shape[0], N))
for i_s, s0 in enumerate(S):
    for i_n in range(N):
        HS = np.zeros(T+1).astype(int)
        HR = np.zeros(T).astype(int)
        HA = np.zeros(T).astype(int)
        HG = np.zeros(T).astype(int)
        HV = np.zeros(T).astype(int)
        HS[0] = s0
        for t in range(T):
            HA[t] = np.random.choice(A, p = pi[np.where(S == HS[t])][0])
            HS[t+1] = np.random.choice(S, p = P[np.where(A == HA[t]), np.where(S == HS[t]), :][0][0])
            HR[t] = R[np.where(A == HA[t]), np.where(S == HS[t])]
        for t in range(0, T, 1):
            HG[T-1-t] = discountedSum(HR[T-1-t:T], d)
        G_[:, i_s, i_n] = HG
    for t in range(T-1):
        V[t, i_s] = np.mean(G_[t, i_s, :])           
print(V[0, :])

[708.9485 706.222  803.248 ]
